In [1]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
from ddeint import ddeint

from scipy.sparse import random
from scipy import stats
import math
import scipy.io
import seaborn as sns
import pandas as pd
from statistics import mean

### Lyapunov exponent for Mackey-Glass equation

In [2]:
def generate_MG_data(init_val, tau, a, b, n, times):
    def MackeyGlass(z, t):
        return (a * z(t - tau)) / (1 + z(t - tau)**n) - b*z(t)

    def initial(t):
        return init_val
    
    start, stop, step_size = times
    time_space = np.arange(start, stop, step_size)

    z_solution_new = ddeint(MackeyGlass, initial, time_space)
    z_solution_new  = [ data_pt[0] for data_pt in z_solution_new[1:] ]
    z_solution_new.insert(0, init_val)
    z_solution_new = [ np.tanh(unshifted_val-1) for unshifted_val in z_solution_new ]
    
    return np.array(z_solution_new)

tau = 17
n = 10
a = 0.2
b = 0.1

init_val = 1.2

delta = 10**(-6)
times = (0, 2000, 1)

MG_traject_ref = generate_MG_data(init_val, tau, a, b, n, times)
MG_traject_perturbed = generate_MG_data(init_val+delta, tau, a, b, n, times)

d1 = np.linalg.norm(MG_traject_ref[1600:1617] - MG_traject_perturbed[1600:1617])
d2 = np.linalg.norm(MG_traject_ref[1800:1817] - MG_traject_perturbed[1800:1817])

ld = np.log(d2/d1)/200

print("Lyapunov exponent for original Mackey-Glass is ", ld)

/Users/hannah/Library/Python/3.8/lib/python/site-packages/ddeint/ddeint.py:145: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([g(tt[0])] + results)


Lyapunov exponent for original Mackey-Glass is  0.0053078773631523015


### ESN helper functions - as lists, without data generator

In [3]:
# Reservoir generating functions
def gen_matrix(shape, sparsity, sd=1, mean=0, loc_seed=100, val_seed=100, pdf="gaussian", seeded=True):
    
    def seeded_rvs_gauss(array_len):
            return stats.norm(loc=mean, scale=sd).rvs(random_state = val_seed, size=array_len)

    def seeded_rvs_uniform(array_len):
        return stats.uniform(loc=mean, scale=sd).rvs(random_state = val_seed, size=array_len)

    m = shape[0]
    n = shape[1]

    if seeded == True:
        
        if pdf == "gaussian":
            M = random(m, n, density=sparsity, random_state=loc_seed, data_rvs=seeded_rvs_gauss).A
            return M

        if pdf == "uniform":
            M = random(m, n, density=sparsity, random_state=loc_seed, data_rvs=seeded_rvs_uniform).A
            return M

        if pdf == "ones":
            M = random(m, n, density = sparsity, random_state=loc_seed, data_rvs=np.ones).A
            return M
        else: 
            print("No such pdf")
            
    elif seeded == False:
        
        if pdf == "gaussian":
            unseeded_rvs = stats.norm(loc=mean, scale=sd).rvs
            M = random(m, n, density=sparsity, data_rvs=unseeded_rvs).A
            return M

        if pdf == "uniform":
            unseeded_rvs = stats.uniform(loc=mean, scale=sd).rvs
            M = random(m, n, density=sparsity, data_rvs=unseeded_rvs).A
            return M

        if pdf == "ones":
            M = random(m, n, density = sparsity, data_rvs=np.ones).A
            return M
        else: 
            print("No such pdf")
            
    else:
        print("Seeded was neither true nor false")

def spectral_radius(M):
    max_abs_eigenvalue = -1
    eigenvalues, eigenvectors = np.linalg.eig(M)
    for eigenvalue in eigenvalues:
        if abs(eigenvalue) > max_abs_eigenvalue:
            max_abs_eigenvalue = abs(eigenvalue)
    return max_abs_eigenvalue

def spectral_radius_matrix(M, desired_spec_rad):
    M_sr = spectral_radius(M)
    if M_sr == 0:
        return M
    else:
        M = M*(desired_spec_rad/M_sr)
        return M
    
# ESN equations
def sigma(value):
    return np.tanh(value)

def state(x_prev, z_curr):
    z_curr = np.atleast_2d(z_curr)
    x_curr = sigma(np.matmul(A, x_prev) + gamma*np.matmul(C, z_curr) + s*zeta)
    return x_curr

def observation(x_curr, w):
    z_next = np.matmul(np.transpose(w), x_curr)
    return z_next

def listening(training_data, x_0, A, gamma, C, s, zeta):
    state_dict = {'all_states': None,
                  'last_state': None, 
                  'input_data': None}
    
    T = len(training_data)
    
    for t in range(0, T):
        if t == 0:
            x_curr = x_0
            X = np.array(x_curr)
            z_curr = training_data[t]
            Z = np.atleast_2d(np.array([z_curr]))
        else:
            x_curr = state(x_curr, z_curr)
            X = np.column_stack((X, x_curr))
            z_curr = training_data[t]
            Z = np.column_stack((Z, z_curr))
            
    x_curr = state(x_curr, z_curr)
            
    state_dict['last_state'] = x_curr
    state_dict['all_states'] = X
    state_dict['input_data'] = Z
    
    return state_dict
    
def regression_sol(ld, state_dict, N):
    X = state_dict['all_states']
    Z = state_dict['input_data']
    
    X = X[:, 1000:]
    Z = Z[:, 1000:]

    X_transpose = X.transpose()
    Z_transpose = Z.transpose()
    XZ_transpose = np.matmul(X, Z_transpose)
    
    inverse_term = np.linalg.inv(np.matmul(X, X_transpose) + ld*np.identity(N))
    W_best = np.matmul(inverse_term, XZ_transpose)
    
    return W_best

def prediction(state_dict, weight, testing_data, training_data, A, gamma, C, s, zeta, platt_err):
    prediction_dict = {'testing_error': None,
                       'z_actuals': None,
                       'z_predictions': None}
    
    z_actuals = []
    z_predictions = []
    testing_error = 0

    T_bar = len(testing_data)
    T = len(training_data)
    
    weight_sum = 0
    x_prev = state_dict.get('last_state')
    for t_bar in range(0, T_bar):
        z_predict = observation(x_prev, weight)[0][0]
        x_prev = state(x_prev, z_predict)
        z_predictions.append(z_predict)
        
        z_actual = testing_data[t_bar]
        z_actuals.append(z_actual)
        
        if platt_err == True:
            testing_error = testing_error + ((z_predict - z_actual)**2)*np.exp(-((t_bar-T)/T_bar))
            weight_sum = weight_sum + np.exp(-((t_bar-T)/T_bar))
        else:    
            testing_error = testing_error + (z_predict - z_actual)**2
            weight_sum = weight_sum + 1
 
    prediction_dict['testing_error'] = testing_error/weight_sum
    prediction_dict['z_actuals'] = z_actuals
    prediction_dict['z_predictions'] = z_predictions        
        
    return prediction_dict

def training_error(state_dict, weight, training_data):
    training_error_dict = {'training_error': None,
                           'z_actuals': None,
                           'z_predictions': None}
    
    z_actuals = []
    z_predictions = []
    
    X = state_dict.get('all_states')
    X = X[:, 1000:]
    
    Z = state_dict.get('input_data')
    Z = Z[:, 1000:]
    
    training_error = 0
    T = len(training_data)
    
    for t in range(0, T-1000):
        x_prev = X[:, t]
        z_predict = observation(x_prev, weight)[0]
        
        z_predictions.append(z_predict)
        z_actual = Z[0, t]
        z_actuals.append(z_actual)
        training_error = training_error + (z_predict - z_actual)**2
        
    training_error = training_error/(T-1000)
    
    training_error_dict['training_error'] = training_error
    training_error_dict['z_actuals'] = z_actuals
    training_error_dict['z_predictions'] = z_predictions
        
    return training_error_dict

# Plotting functions
def state_plot(state_dict, plotwith_init, node=0):
    X = state_dict.get('all_states')
    if plotwith_init == True:
        state_plot, state_ax = plt.subplots(figsize=(20,5))
        state_ax.plot(X[node][:])
        state_ax.set_title('Plot of States at node {}'.format(node))
        state_ax.set_xlabel('time')
        state_ax.set_ylabel('state of node {}'.format(node))
        
        return (np.amin(X[node][:]), np.amax(X[node][:]))
    
    if plotwith_init == False:
        state_plot, state_ax = plt.subplots(figsize=(20,5))
        state_ax.plot(X[node][1000:])
        state_ax.set_title('Plot of States at node {}'.format(node))
        state_ax.set_xlabel('time')
        state_ax.set_ylabel('state of node {}'.format(node))
    
        return (np.amin(X[node][1000:]), np.amax(X[node][1000:]))
                
def hist_accuracy_plot(actuals, predictions, with_bars=False):
    if with_bars == False:
        sns.kdeplot(actuals, label='actual', shade=True, color='red')
        sns.kdeplot(predictions, label='prediction', shade=True, color='skyblue')
        plt.legend()
        
    if with_bars == True:
        sns.histplot(actuals, label='actual', color='red', kde=True)
        sns.histplot(predictions, label='prediction', color='skyblue', kde=True)
        plt.legend()

### Run tuned ESN on data set to output reference trajectory

In [4]:
times = (0, 5000, 1)
data_set = generate_MG_data(init_val, tau, a, b, n, times) 

# ESN for reference trajectory
training_data = data_set[0:3000]
testing_data = data_set[3000:5000]

Training error:  7.405284780331518e-08
Testing error:  0.06455036448521943


In [5]:
np.random.seed(100)

N = 1000
ld = 10**(-14)
C = gen_matrix(shape=(N,1), sparsity=1, sd=2, mean=-1, pdf="uniform", seeded=False)
gamma = 0.8
A = gen_matrix(shape=(N,N), sparsity=0.01, sd=2, mean=-1, pdf="uniform", seeded=False)
spec_rad = 1.2
A = spectral_radius_matrix(A, spec_rad)
s = 0.2
zeta = gen_matrix(shape=(N,1), sparsity=1, pdf="ones", seeded=False)
x_0 = np.zeros(shape=(N,1), dtype=float)

state_dict = listening(training_data, x_0, A, gamma, C, s, zeta)
weight_result = regression_sol(ld, state_dict, N) 
training = training_error(state_dict, weight_result, training_data)
predicting = prediction(state_dict, weight_result, testing_data, training_data, A, gamma, C, s, zeta, platt_err=True)

print("Training error: ", training['training_error'])
print("Testing error: ", predicting['testing_error'])

z_actuals = training['z_actuals'] + predicting['z_actuals']
z_predictions = training['z_predictions'] + predicting['z_predictions']

forecast_plot, forecast_ax = plt.subplots(figsize=(30, 10))
forecast_ax.axvline(len(training_data), color='black', linestyle="--")
forecast_ax.plot(range(1001, len(z_actuals)+1001), z_actuals, label='actual', lw=1.2)
forecast_ax.plot(range(1001, len(z_predictions)+1001), z_predictions, label='prediction', lw=1.2)
forecast_ax.set_title('Mackey-Glass system forecast (testing only)')
forecast_ax.set_xlabel('time')
forecast_ax.legend()

In [7]:
ESN_traject_ref = np.array(predicting['z_predictions'])   # only the prediction portion

trial = 0
ld_ls = []
state_dict_perturbed = state_dict.copy()

while trial <= 100:
    
    perturbation = 10**(-6) * np.random.uniform(low=0, high=1, size=(N,1))
    state_dict_perturbed['last_state'] = state_dict.copy()['last_state'] + perturbation
    predicting_perturbed = prediction(state_dict_perturbed, weight_result, testing_data, training_data,
                                      A, gamma, C, s, zeta, platt_err=True)
    
    ESN_traject_perturbed = np.array(predicting_perturbed['z_predictions'])
    
    d1 = np.linalg.norm(perturbation)
    d2 = np.linalg.norm(ESN_traject_ref[400:417] - ESN_traject_perturbed[400:417])
    
    ld = np.log(d2/d1)/400
    ld_ls.append(ld)
    
    trial = trial + 1

print("Lyapunov exponent for ESN is ", mean(ld_ls))

Lyapunov exponent for ESN is  0.003794274745935891


In [10]:
ESN_traject_ref = np.array(predicting['z_predictions'])   # only the prediction portion

trial = 0
ld_ls = []
state_dict_perturbed = state_dict.copy()

while trial <= 100:
    
    perturbation = 10**(-3) * np.random.uniform(low=0, high=1, size=(N,1))
    state_dict_perturbed['last_state'] = state_dict.copy()['last_state'] + perturbation
    predicting_perturbed = prediction(state_dict_perturbed, weight_result, testing_data, training_data,
                                      A, gamma, C, s, zeta, platt_err=True)
    
    ESN_traject_perturbed = np.array(predicting_perturbed['z_predictions'])
    
    d1 = np.linalg.norm(perturbation)
    d2 = np.linalg.norm(ESN_traject_ref[400:417] - ESN_traject_perturbed[400:417])
    
    ld = np.log(d2/d1)/400
    ld_ls.append(ld)
    
    trial = trial + 1

print("Lyapunov exponent for ESN is ", mean(ld_ls))

Lyapunov exponent for ESN is  0.003436736162740953
